In [ ]:
# https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

#git clone https://github.com/matthiasplappert/keras-rl.git
#cd keras-rl
#python setup.py install

#pip install gym

In [1]:
import gym

env = gym.make("Taxi-v2").env

env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [2]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [3]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()


State: 328
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
#reward for each Q value (state, action) pair
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [6]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [7]:
q_table.size

3000

In [8]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 34.1 s, sys: 9.48 s, total: 43.6 s
Wall time: 36 s


In [9]:
q_table[328]

array([ -2.39222039,  -2.27325184,  -2.4092054 ,  -2.36191545,
       -10.60295506, -10.67791978])

In [10]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
#episodes = 100
episodes = 1

for _ in range(episodes):
    
    state = env.reset()
    print(state)
    env.render()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        print(state, action, reward, done, info)
        env.render()
        
        print("reward=", reward)
        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

251
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

231 3 -1 False {'prob': 1.0}
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
reward= -1
211 3 -1 False {'prob': 1.0}
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
reward= -1
311 0 -1 False {'prob': 1.0}
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
reward= -1
411 0 -1 False {'prob': 1.0}
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
reward= -1
419 4 -1 False {'prob': 1.0}
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
reward= -1
319 1 -1 False {'prob': 1.0}
+---------+
|R: | : :G|
| : : : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)
reward= -1
219 1 -1 False {'prob': 1.0}
+---------+
|R: | : :G|
| : : : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+------

In [11]:
dir(env)

['P',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'action_space',
 'close',
 'decode',
 'desc',
 'encode',
 'isd',
 'lastaction',
 'locs',
 'metadata',
 'nA',
 'nS',
 'np_random',
 'observation_space',
 'render',
 'reset',
 'reward_range',
 's',
 'seed',
 'spec',
 'step',
 'unwrapped']

In [12]:
q_table

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -2.27325184,  -2.1220864 ,  -2.27325183,  -2.1220864 ,
         -1.870144  , -11.12208632],
       [ -1.87014392,  -1.45024004,  -1.87014392,  -1.45024001,
         -0.7504    , -10.45023782],
       ...,
       [ -1.05877048,   0.41599994,  -1.11134602,  -1.26514   ,
         -6.02578884,  -4.75206171],
       [ -2.15844556,  -2.12206592,  -2.16796172,  -2.12206434,
         -5.0384112 ,  -5.74031226],
       [  2.82765163,   1.34467635,   3.9377386 ,  11.        ,
         -2.75269457,  -2.36988609]])